In [1]:
from comet_ml import Experiment
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import importlib
import utils
importlib.reload(utils)
from utils import DataLoader, load_data, load_model, print_performances, bleu_fromdatalist
from train import train_epoch, eval_epoch
import sentencepiece as spm
from tqdm import tqdm
from model import Transformer
import Constant as Constant
import time
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
experiment = Experiment(api_key='tG1tXUexkJuIlhnNFNCzg4JxO',project_name="transformer_deen_multi30k")

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/sosuke115/transformer-deen-multi30k/bc62f125f06f4293ab1902c1b6859fbc



In [3]:
sp = spm.SentencePieceProcessor(model_file='8k_sp.model')

In [4]:
vocabs = [[sp.id_to_piece(idx), idx] for idx in range(sp.get_piece_size())]

In [5]:
train_X, train_Y = load_data("multi30k/train.de.tok", "multi30k/train.en.tok", sp)
valid_X, valid_Y = load_data("multi30k/val.de.tok", "multi30k/val.en.tok", sp)
test_X, test_Y = load_data("multi30k/test2016.de.tok", "multi30k/test2016.en.tok", sp)

29000it [00:01, 19684.48it/s]
1014it [00:00, 20424.75it/s]
1000it [00:00, 9452.49it/s]


In [6]:
# Training hyperparameters
num_epochs = 30
learning_rate = 3e-4
batch_size = 32
gradient_accumulation_steps = 1

# Model hyperparameters

model_args = {
    "embedding_size": 512,
    'src_vocab_size': len(vocabs),
    'tgt_vocab_size': len(vocabs),
    "src_pad_idx" : Constant.PAD,
    'num_heads': 8,
    'num_encoder_layers': 3,
    'num_decoder_layers': 3,
    "forward_expansion": 4,
    'dropout': 0.1,
    "max_len" : 100,
    "device" : device
}

In [7]:
model = Transformer(
    model_args["embedding_size"],
    model_args["src_vocab_size"],
    model_args["tgt_vocab_size"],
    model_args["src_pad_idx"],
    model_args["num_heads"],
    model_args["num_encoder_layers"],
    model_args["num_decoder_layers"],
    model_args["forward_expansion"],
    model_args["dropout"],
    model_args["max_len"],
    model_args["device"],
).to(device)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, factor=0.1, patience=10, verbose=True
)

In [8]:
train_dataloader = DataLoader(
    train_X, train_Y, batch_size, device
    )
valid_dataloader = DataLoader(
    valid_X, valid_Y, batch_size, device,
    shuffle=False
    )

In [9]:
valid_losses = []
model_path = "results/model_hard_c.chkpt"
for epoch_i in range(num_epochs):
    print('[ Epoch', epoch_i, ']')

    start = time.time()
    train_loss, train_accu = train_epoch(model, train_dataloader, optimizer, scheduler, device)
    print_performances('Training', train_loss, train_accu, start)

    start = time.time()
    valid_loss, valid_accu = eval_epoch(model, valid_dataloader, device)
    print_performances('Validation', valid_loss, valid_accu, start)
    valid_losses += [valid_loss]
    checkpoint = {'epoch': epoch_i, 'settings': model_args, 'state_dict': model.state_dict(), "optimizer": optimizer.state_dict()}
    
    if valid_loss <= min(valid_losses):
        torch.save(checkpoint, model_path)
        print('    - [Info] The checkpoint file has been updated.')
    metrics = {
                'Train. Loss':train_loss,
                'Train. Acc':train_accu,
                'Valid. Loss': valid_loss,
                'Valid. Acc':valid_accu
                
    }
    print(metrics)
    num_step = int(epoch_i) + 1
    experiment.log_metrics(metrics, step=num_step)

  - (Training)   : 0it [00:00, ?it/s]

[ Epoch 0 ]


  - (Validation) : 0it [00:00, ?it/s]      

  - (Training)   ppl:  29.11856, accuracy: 43.798 %, elapse: 0.848 min


  - (Validation) ppl:  12.97707, accuracy: 54.100 %, elapse: 0.009 min


...

  - (Training)   : 0it [00:00, ?it/s]

  - (Validation) ppl:  6.98789, accuracy: 69.757 %, elapse: 0.007 min
{'Train. Loss': 0.4354544243345581, 'Train. Acc': 0.8664793056716013, 'Valid. Loss': 1.9441785490722245, 'Valid. Acc': 0.6975700095403783}
[ Epoch 29 ]


  - (Validation) : 0it [00:00, ?it/s]      

  - (Training)   ppl:  1.52471, accuracy: 86.983 %, elapse: 0.872 min


  - (Validation) ppl:  7.13042, accuracy: 69.701 %, elapse: 0.007 min
{'Train. Loss': 0.4218043732914104, 'Train. Acc': 0.8698254184639863, 'Valid. Loss': 1.9643699029016604, 'Valid. Acc': 0.6970088108199113}


In [12]:
checkpoint = torch.load(model_path, map_location=device)
model = load_model(checkpoint, device)

[Info] Trained model state loaded.


In [13]:
score = bleu_fromdatalist(test_X[:10], test_Y[:10], model, sp, device)
print(f"Bleu score {score * 100:.2f}")

10it [00:02,  4.66it/s]

Bleu score 33.42
